# Predicting Hits on Spotify using Machine Learning

In this era of digital music, understanding the dynamics of song popularity is more intriguing than ever! With millions of songs available at our fingertips, we were curious to leverage machine learning to launch a data-driven exploration into what makes songs hits. For our experiments, we'll take a closer look into Spotify, one of the largest music streaming platforms in the world, and aim to predict song popularity through a variety of different modeling and analysis techniques.

## Defining a Library

First, lets import all relevant libraries needed for data analysis, modeling, and visualizations.

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Fetching Data

For our experiments, we'll use a Kaggle dataset sourced by developer Nidula Elgiryewithana, a comprehensive list of 2023 songs with an extensive insights on musical attributes, popularity, relevant metrics, and more. First, we read and load the data from its original CSV format.

In [ ]:
data = pd.read_csv('/data/spotify-2023.csv', encoding='latin-1')
data.head()

## Data Exploration

Given the data, now let's take a closer look the features provided, their data-types, and other attributes which may need to adjusted or cleaned prior to experimentation.